In [1]:
from tdc.multi_pred import DTI
import re
import torch
import numpy as np
from PyDRP.Data.features.targets import MinMaxScaling
from PyDRP.Data.features.drugs import GraphCreator
from PyDRP.Data.features.proteins import BertProteinFeaturizer
from PyDRP.Data import DTIDatasetManager, TDCDTIWrapper
import pandas as pd

In [2]:
manager = DTIDatasetManager(TDCDTIWrapper(DTI(name = 'BindingDB_Kd')),
                 MinMaxScaling(),
                 GraphCreator(),
                 BertProteinFeaturizer(),
                 partition_column="PROTEIN_ID")

Found local copy...
Loading...
Done!
Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
protein_dict = manager.get_proteins()

In [5]:
drug_dict = manager.get_drugs()